In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
import notebook_helper
import imodelsx.process_results
import sys
tqdm.pandas()
from mprompt.modules.fmri_module import get_test_ngrams, get_roi
sys.path.append('../experiments/')
# results_dir = '../results/feb12_fmri_sweep/'
results_dir = '/home/chansingh/mntv1/mprompt/feb12_fmri_sweep_gen_template1/'

r = imodelsx.process_results.get_results_df(results_dir, use_cached=True)
r['top_ngrams_test'] = r.apply(lambda row: get_test_ngrams(voxel_num_best=row.module_num)[:25], axis=1)
r['roi'] = r.apply(lambda row: get_roi(voxel_num_best=row.module_num), axis=1)
r['explanation_init_ngrams'] = r['explanation_init_ngrams'].apply(lambda x: x[:25])
r.to_pickle('../results/results_fmri.pkl')

In [40]:
import bert_score
from typing import List
def similarity(explanation: str, top_ngrams_test: List[str]):
    n = len(top_ngrams_test)
    P, R, F1 = bert_score.score(
        [explanation] * n,
        top_ngrams_test,
        model_type='microsoft/deberta-xlarge-mnli'
    )
    bscore = F1.detach().cpu().numpy().mean()
    return bscore
    
r['expl_test_bert_score'] = r.progress_apply(lambda row: similarity(row['top_explanation_init_strs'], row['top_ngrams_test'].tolist()), axis=1)

Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'pooler.dense.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  6%|▌         | 11/200 [00:58<18:11,  5.78s/it]

### Visualize table

In [ ]:
tab = (
    r
    .sort_values(by='top_score_synthetic', ascending=False)
    .filter([
        'module_num', 'roi', 'top_score_synthetic', 'fmri_test_corr',
        'top_explanation_init_strs', 'explanation_init_ngrams',
        'top_ngrams_test',
        'explanation_init_strs',
    ])
    .round(3)
    .rename(
        columns={
            'top_explanation_init_strs': 'Explanation',
            'roi': 'ROI',
            'explanation_init_ngrams': 'Top ngrams (module)',
            'module_num': 'Voxel',
            'top_ngrams_test': 'Top ngrams (test)',
            'top_score_synthetic': 'Synthetic score',
            'fmri_test_corr': 'Correlation (test)',
            'explanation_init_strs': 'Explanation candidates',
        }
    )
)
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    display(
        tab.head(2)
    )
    tab.to_csv('../results/results_fmri.csv', index=False)

In [ ]:
sns.lmplot(x='top_score_synthetic', y='fmri_test_corr', data=r)
plt.show()